In [2]:
import numpy as np
import lightkurve as lk
import matplotlib.pyplot as plt
from scipy import stats
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
import os

from flare_timestamps import get_timestamps
from flare_timestamps import get_rotation_period

#these are used for the second half of the dataset (superflares_2.txt) because the dataset had to be split and processed in two parts

from flare_timestamps2 import get_timestamps_2
from flare_timestamps2 import get_rotation_period_2

In order to get the standard deviation of the flux in different time intervals before the flare, the time intervals have to be defined. For that the amount of datapoints in each interval have to be calculated first. Then the flux values for these datapoints can be put into separate arrays and then the standard deviation can be calculated for each array separately.

In [ ]:
#create new folders for the different standard deviations
os.mkdir('std_1d')
os.mkdir('std_5d')
os.mkdir('std_10d')
os.mkdir('std_20d')
os.mkdir('std_40d')

In [ ]:
#recreate the same code as above but for all target names
target_names= np.unique(np.loadtxt('superflares.txt', dtype='str', usecols=(0)))
input_file = 'superflares.txt'


for target_name in target_names:
    all_timestamps = get_timestamps(target_name, input_file)
    rotation_period = get_rotation_period(target_name, input_file)

    try:
        search_result = lk.search_lightcurve(f'KIC {target_name}', mission='Kepler', cadence='long')
        lc_collection = search_result.download_all()
        lc = np.array((lc_collection.stitch().flux) - 1)
        t = np.array(lc_collection.stitch().time.value)
    except:
        print('Target not found')
        exit()

    timestamps = []
    for element in all_timestamps:
        element = float(element) + float(2400000)
        timestamp = np.round(element - 2454833, 2)
        timestamps.append(timestamp)

    std_1d= []
    std_5d= []
    std_10d= []
    std_20d= []
    std_40d= []
    for element in timestamps:


        #round t to 2 decimals
        t = np.round(t, 2)
        el = int(element - 54833)
        index = np.where(t == element)
        # # print(lc[index])

        # flux.append(lc[index])

        #find the flux value one index before the flare
        start_index = index[0]
        # print(lc[start_index - 1])


        #calculate the mean time difference between two data points
        mean_time_diff = np.mean(np.diff(t))

        #from the mean time difference, calculate how many data points are in 1, 5, 10, 20 and 40 days
        one_day = int(1/mean_time_diff)
        five_days = int(5/mean_time_diff)
        ten_days = int(10/mean_time_diff)
        twenty_days = int(20/mean_time_diff)
        fourty_days = int(40/mean_time_diff)

        #make a separate array saving only the flux values one day before the flare
        lc_1d_before = lc[el- one_day:el]
        lc_5d_before = lc[el-five_days:el]
        lc_10d_before = lc[el-ten_days:el]
        lc_20d_before = lc[el-twenty_days:el]
        lc_40d_before = lc[el-fourty_days:el]

        #erase all nan values from the arrays
        lc_1d_before = lc_1d_before[~np.isnan(lc_1d_before)]
        lc_5d_before = lc_5d_before[~np.isnan(lc_5d_before)]
        lc_10d_before = lc_10d_before[~np.isnan(lc_10d_before)]
        lc_20d_before = lc_20d_before[~np.isnan(lc_20d_before)]
        lc_40d_before = lc_40d_before[~np.isnan(lc_40d_before)]

        #calculate the standard deviation of the arrays above
        std_1d_before = np.std(lc_1d_before)
        std_5d_before = np.std(lc_5d_before)
        std_10d_before = np.std(lc_10d_before)
        std_20d_before = np.std(lc_20d_before)
        std_40d_before = np.std(lc_40d_before)


        std_1d.append(std_1d_before)
        std_5d.append(std_5d_before)
        std_10d.append(std_10d_before)
        std_20d.append(std_20d_before)
        std_40d.append(std_40d_before)

    #for each target name, write the standard deviations in separate text files and save them in the std_1d, std_5d, std_10d, std_20d and std_40d folders
    np.savetxt(f'std_1d/std_1d_{target_name}.txt', std_1d, fmt='%s')
    np.savetxt(f'std_5d/std_5d_{target_name}.txt', std_5d, fmt='%s')
    np.savetxt(f'std_10d/std_10d_{target_name}.txt', std_10d, fmt='%s')
    np.savetxt(f'std_20d/std_20d_{target_name}.txt', std_20d, fmt='%s')
    np.savetxt(f'std_40d/std_40d_{target_name}.txt', std_40d, fmt='%s')
    print(target_name, 'done')

create suitable arrays for the standard deviation and bolometric energy so that they can be plotted against each other.

In [3]:

std_1d=np.loadtxt('std_1d.txt')
std_1d_2=np.loadtxt('std_1d_2.txt')
std_5d=np.loadtxt('std_5d.txt')
std_5d_2= np.loadtxt('std_5d_2.txt')
std_10d=np.loadtxt('std_10d.txt')
std_10d_2=np.loadtxt('std_10d_2.txt')
std_20d=np.loadtxt('std_20d.txt')
std_20d_2=np.loadtxt('std_20d_2.txt')
std_40d=np.loadtxt('std_40d.txt')
std_40d_2=np.loadtxt('std_40d_2.txt')
bolom= np.loadtxt('superflares.txt', usecols=13)
bolom1= np.loadtxt('superflares.txt', usecols=13)
bolom5= np.loadtxt('superflares.txt', usecols=13)
bolom10= np.loadtxt('superflares.txt', usecols=13)
bolom20= np.loadtxt('superflares.txt', usecols=13)
bolom40= np.loadtxt('superflares.txt', usecols=13)

#match the number of bolometric energies with the number of standard deviations, then combine them in one array with two columns and then remove the rows with nan values
if len(bolom1) > len(std_1d):
    bolom1 = bolom1[0:len(std_1d)]
elif len(std_1d) > len(bolom1):
    std_1d = std_1d[0:len(bolom1)]
else:
    pass

if len(bolom5) > len(std_5d):
    bolom5 = bolom5[0:len(std_5d)]
elif len(std_5d) > len(bolom5):
    std_5d = std_5d[0:len(bolom5)]
else:
    pass

if len(bolom) > len(std_5d_2):
    bolom5_2 = bolom[0:len(std_5d_2)]
elif len(std_5d_2) > len(bolom):
    std_5d_2 = std_5d_2[0:len(bolom)]
else:
    pass

#repeat the same for std_10d_2, std_20d_2 and std_40d_2
if len(bolom) > len(std_10d_2):
    bolom10_2 = bolom[0:len(std_10d_2)]
elif len(std_10d_2) > len(bolom):
    std_10d_2 = std_10d_2[0:len(bolom)]
else:
    pass

if len(bolom) > len(std_20d_2):
    bolom20_2 = bolom[0:len(std_20d_2)]
elif len(std_20d_2) > len(bolom):
    std_20d_2 = std_20d_2[0:len(bolom)]
else:
    pass

if len(bolom) > len(std_40d_2):
    bolom40_2 = bolom[0:len(std_40d_2)]
elif len(std_40d_2) > len(bolom):
    std_40d_2 = std_40d_2[0:len(bolom)]
else:
    pass

if len(bolom10) > len(std_10d):
    bolom10 = bolom10[0:len(std_10d)]
elif len(std_10d) > len(bolom10):
    std_10d = std_10d[0:len(bolom10)]
else:
    pass

if len(bolom20) > len(std_20d):
    bolom20 = bolom20[0:len(std_20d)]
elif len(std_20d) > len(bolom20):
    std_20d = std_20d[0:len(bolom20)]
else:
    pass

if len(bolom40) > len(std_40d):
    bolom40 = bolom40[0:len(std_40d)]
elif len(std_40d) > len(bolom40):
    std_40d = std_40d[0:len(bolom40)]
else:
    pass

combined1 = np.column_stack((bolom1, std_1d))
combined5 = np.column_stack((bolom5, std_5d))
combined10 = np.column_stack((bolom10, std_10d))
combined20 = np.column_stack((bolom20, std_20d))
combined40 = np.column_stack((bolom40, std_40d))
combined5_2 = np.column_stack((bolom5_2, std_5d_2))
combined10_2 = np.column_stack((bolom10_2, std_10d_2))
combined20_2 = np.column_stack((bolom20_2, std_20d_2))
combined40_2 = np.column_stack((bolom40_2, std_40d_2))

combined1 = combined1[~np.isnan(combined1).any(axis=1)]
combined5 = combined5[~np.isnan(combined5).any(axis=1)]
combined10 = combined10[~np.isnan(combined10).any(axis=1)]
combined20 = combined20[~np.isnan(combined20).any(axis=1)]
combined40 = combined40[~np.isnan(combined40).any(axis=1)]
combined5_2 = combined5_2[~np.isnan(combined5_2).any(axis=1)]
combined10_2 = combined10_2[~np.isnan(combined10_2).any(axis=1)]
combined20_2 = combined20_2[~np.isnan(combined20_2).any(axis=1)]
combined40_2 = combined40_2[~np.isnan(combined40_2).any(axis=1)]

#combine the two stacks combined5 and combined5_2
combined5 = np.concatenate((combined5, combined5_2), axis=0)
combined10 = np.concatenate((combined10, combined10_2), axis=0)
combined20 = np.concatenate((combined20, combined20_2), axis=0)
combined40 = np.concatenate((combined40, combined40_2), axis=0)

#save the combined arrays in separate text files
np.savetxt('combined1_std.txt', combined1, fmt='%s')
np.savetxt('combined5_std.txt', combined5, fmt='%s')
np.savetxt('combined10_std.txt', combined10, fmt='%s')
np.savetxt('combined20_std.txt', combined20, fmt='%s')
np.savetxt('combined40_std.txt', combined40, fmt='%s')

C:\Users\sophi\AppData\Local\Temp\ipykernel_13728\3491390316.py:2: UserWarning: loadtxt: input contained no data: "std_1d_2.txt"
  std_1d_2=np.loadtxt('std_1d_2.txt')


In [4]:
#remove all nan values and zeros from the arrays
combined1 = combined1[~np.isnan(combined1).any(axis=1)]
combined5 = combined5[~np.isnan(combined5).any(axis=1)]
combined1 = combined1[~(combined1 == 0).any(axis=1)]
combined5 = combined5[~(combined5 == 0).any(axis=1)]
combined10 = combined10[~np.isnan(combined10).any(axis=1)]
combined20 = combined20[~np.isnan(combined20).any(axis=1)]
combined40 = combined40[~np.isnan(combined40).any(axis=1)]
combined10= combined10[~(combined10 == 0).any(axis=1)]
combined20 = combined20[~(combined20 == 0).any(axis=1)]
combined40 = combined40[~(combined40 == 0).any(axis=1)]

Calculate all necessary statistical values such as the correlation coefficient, and the p-values.

In [5]:

#calculate the linear regressions
slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(combined1[:,0], combined1[:,1])
slope5, intercept5, r_value5, p_value5, std_err5 = stats.linregress(combined5[:,0], combined5[:,1])
slope10, intercept10, r_value10, p_value10, std_err10 = stats.linregress(combined10[:,0], combined10[:,1])
slope20, intercept20, r_value20, p_value20, std_err20 = stats.linregress(combined20[:,0], combined20[:,1])
slope40, intercept40, r_value40, p_value40, std_err40 = stats.linregress(combined40[:,0], combined40[:,1])

#print the r_values
print('r_value for 1 day:', r_value1)
print('r_value for 5 days:', r_value5)
print('r_value for 10 days:', r_value10)
print('r_value for 20 days:', r_value20)
print('r_value for 40 days:', r_value40)


#calculate the standard errors of the r_values
std_err1 = np.sqrt((1 - r_value1**2)/(len(combined1)-2))
std_err5 = np.sqrt((1 - r_value5**2)/(len(combined5)-2))
std_err10 = np.sqrt((1 - r_value10**2)/(len(combined10)-2))
std_err20 = np.sqrt((1 - r_value20**2)/(len(combined20)-2))
std_err40 = np.sqrt((1 - r_value40**2)/(len(combined40)-2))

#print the standard errors
print('Standard error for 1 day:', std_err1)
print('Standard error for 5 days:', std_err5)
print('Standard error for 10 days:', std_err10)
print('Standard error for 20 days:', std_err20)
print('Standard error for 40 days:', std_err40)


#calculate the the test statistic (z-score) and then the p-value
z_score1 = (r_value1 - r_value5)/np.sqrt(std_err1**2 + std_err5**2)
p_value1 = stats.norm.sf(abs(z_score1))*2

z_score40= (r_value1-r_value40)/np.sqrt(std_err1**2 + std_err40**2)
p_value40 = stats.norm.sf(abs(z_score40))*2

z_score20= (r_value1-r_value20)/np.sqrt(std_err1**2 + std_err20**2)
p_value20 = stats.norm.sf(abs(z_score20))*2

z_score10= (r_value1-r_value10)/np.sqrt(std_err1**2 + std_err10**2)
p_value10 = stats.norm.sf(abs(z_score10))*2

print ('p-value for 1 day:', p_value1)
print ('p-value for 5 days:', p_value5)
print ('p-value for 10 days:', p_value10)
print ('p-value for 20 days:', p_value20)
print ('p-value for 40 days:', p_value40)

r_value for 1 day: 0.4182093486369201
r_value for 5 days: 0.23114831095666674
r_value for 10 days: 0.22544629508470077
r_value for 20 days: 0.2215112890648914
r_value for 40 days: 0.2343407302153954
Standard error for 1 day: 0.023707759521323417
Standard error for 5 days: 0.021870241367539168
Standard error for 10 days: 0.021900297390379315
Standard error for 20 days: 0.02192057818032827
Standard error for 40 days: 0.021853067402714598
p-value for 1 day: 6.651371595881552e-09
p-value for 5 days: 1.9467546964419942e-25
p-value for 10 days: 2.3363941312945964e-09
p-value for 20 days: 1.116294445497686e-09
p-value for 40 days: 1.1800677280326006e-08


A statistical test can also be performed for the number of occurances (the histogram) of the standard deviations. Here only the standard deviations are considered without the bolometric energies of the flares. 

In [6]:
#calculate the mean in each hisogram

mean1= np.mean(combined1[:,1])
print(mean1)

mean5= np.mean(combined5[:,1])
print(mean5)

mean10= np.mean(combined10[:,1])
print(mean10)

mean20= np.mean(combined20[:,1])
print(mean20)

mean40= np.mean(combined40[:,1])
print(mean40)

median1= np.median(combined1[:,1])
print(median1)
median5= np.median(combined5[:,1])
print(median5)
median10= np.median(combined10[:,1])
print(median10)
median20= np.median(combined20[:,1])
print(median20)
median40= np.median(combined40[:,1])
print(median40)

std1= np.std(combined1[:,1])
print(std1)
std5= np.std(combined5[:,1])
print(std5)
std10= np.std(combined10[:,1])
print(std10)
std20= np.std(combined20[:,1])
print(std20)
std40= np.std(combined40[:,1])
print(std40)

z_score1= (combined1[:,1] - mean1)/std1
print(z_score1)
z_score5= (combined5[:,1] - mean5)/std5
print(z_score5)
z_score10= (combined10[:,1] - mean10)/std10
print(z_score10)
z_score20= (combined20[:,1] - mean20)/std20
print(z_score20)
z_score40= (combined40[:,1] - mean40)/std40
print(z_score40)



#create new combined arrays now with the z_scores
combined1 = np.column_stack((combined1[:,0], z_score1))
combined5 = np.column_stack((combined5[:,0], z_score5))
combined10 = np.column_stack((combined10[:,0], z_score10))
combined20 = np.column_stack((combined20[:,0], z_score20))
combined40 = np.column_stack((combined40[:,0], z_score40))

#sort the arrays by the z_score starting with the highest z_score
combined1 = combined1[combined1[:,1].argsort()[::-1]]
combined5 = combined5[combined5[:,1].argsort()[::-1]]
combined10 = combined10[combined10[:,1].argsort()[::-1]]
combined20 = combined20[combined20[:,1].argsort()[::-1]]
combined40 = combined40[combined40[:,1].argsort()[::-1]]

#erase the rows with z_score over 3
combined1 = combined1[combined1[:,1] < 2]
combined5 = combined5[combined5[:,1] < 2]
combined10 = combined10[combined10[:,1] < 2]
combined20 = combined20[combined20[:,1] < 2]
combined40 = combined40[combined40[:,1] < 2]

0.006672579817292518
0.011453248188576477
0.011822796832715799
0.012031210126900555
0.011947132496945989
0.00417043575
0.009824857
0.01045806
0.010868426
0.01140157
0.0070828930446855285
0.009498201513793554
0.009372056016293463
0.00899790711200133
0.008186938900186602
[-0.00888716  1.18346516  1.37057035 ... -0.89213789 -0.88220901
 -0.74810425]
[-0.50183271  0.31009837  0.35874463 ... -1.02466294 -1.01375378
 -1.00552282]
[-0.51299931  0.07676674  0.12199235 ... -1.07300371 -1.07156247
 -1.06279306]
[-0.12530137 -0.18329853 -0.15719212 ... -1.15339974 -1.15174139
 -1.13376552]
[ 0.68775895  0.21471578  0.20469134 ... -1.27493766 -1.27146459
 -1.26107058]


In [7]:


#perform a hypothesis tests such as an Analysis of Variance between all the combined arrays separately, meaning in between combined1 and combined5, then combined5 and combined10, then combined10 and combined20 and finally combined20 and combined40

#perform an Analysis of Variance between combined1 and combined5
f_value, p_value = stats.f_oneway(combined1[:,1], combined5[:,1])
print('p-value for combined1 and combined5:', p_value)

#perform an Analysis of Variance between combined5 and combined10
f_value, p_value = stats.f_oneway(combined5[:,1], combined10[:,1])
print('p-value for combined5 and combined10:', p_value)

#perform an Analysis of Variance between combined10 and combined20
f_value, p_value = stats.f_oneway(combined10[:,1], combined20[:,1])
print('p-value for combined10 and combined20:', p_value)

#perform an Analysis of Variance between combined20 and combined40
f_value, p_value = stats.f_oneway(combined20[:,1], combined40[:,1])
print('p-value for combined20 and combined40:', p_value)

#now perform a Tukey's HSD (Honestly Significant Difference) test between all the arrays
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

#perform normality tests (e.g., Shapiro-Wilk) and Levene's test to see if the requirements are met for the HSD test
from scipy.stats import shapiro

#perform a Shapiro-Wilk test for normality
stat, p = shapiro(combined1[:,1])
print('Statistics=%.3f, p=%.3f' % (stat, p))

#perform a Shapiro-Wilk test for normality
stat, p = shapiro(combined5[:,1])
print('Statistics=%.3f, p=%.3f' % (stat, p))

#perform a Shapiro-Wilk test for normality
stat, p = shapiro(combined10[:,1])
print('Statistics=%.3f, p=%.3f' % (stat, p))

#perform a Shapiro-Wilk test for normality
stat, p = shapiro(combined20[:,1])
print('Statistics=%.3f, p=%.3f' % (stat, p))

#perform a Shapiro-Wilk test for normality
stat, p = shapiro(combined40[:,1])
print('Statistics=%.3f, p=%.3f' % (stat, p))

#perform a Levene's test for homogeneity of variances
from scipy.stats import levene

#perform a Levene's test for homogeneity of variances
stat, p = levene(combined1[:,1], combined5[:,1])
print('Statistics=%.3f, p=%.3f' % (stat, p))

stat, p = levene(combined5[:,1], combined10[:,1])
print('Statistics=%.3f, p=%.3f' % (stat, p))

stat, p = levene(combined10[:,1], combined20[:,1])
print('Statistics=%.3f, p=%.3f' % (stat, p))

stat, p = levene(combined20[:,1], combined40[:,1])
print('Statistics=%.3f, p=%.3f' % (stat, p))

p-value for combined1 and combined5: 0.7309358238477365
p-value for combined5 and combined10: 0.8775395949610205
p-value for combined10 and combined20: 0.6587949614782086
p-value for combined20 and combined40: 0.15942236366078874
Statistics=0.859, p=0.000
Statistics=0.925, p=0.000
Statistics=0.934, p=0.000
Statistics=0.945, p=0.000
Statistics=0.953, p=0.000
Statistics=8.746, p=0.003
Statistics=0.215, p=0.643
Statistics=4.584, p=0.032
Statistics=16.002, p=0.000


In [8]:
#make a pairwwise camparison between the arrays with the Kruskal-Wallis Test
stat, p = stats.kruskal(combined1[:,1], combined5[:,1])
print('Statistics=%.3f, p=%.3f' % (stat, p))

stat, p = stats.kruskal(combined5[:,1], combined10[:,1])
print('Statistics=%.3f, p=%.3f' % (stat, p))

stat, p = stats.kruskal(combined10[:,1], combined20[:,1])
print('Statistics=%.3f, p=%.3f' % (stat, p))

stat, p = stats.kruskal(combined20[:,1], combined40[:,1])
print('Statistics=%.3f, p=%.3f' % (stat, p))

Statistics=1.583, p=0.208
Statistics=0.000, p=0.996
Statistics=0.019, p=0.889
Statistics=0.681, p=0.409


In [9]:
from scipy.stats import kruskal

# Perform the Kruskal-Wallis test
kw_statistic, p_value = kruskal(combined1[:, 1], combined5[:, 1], combined10[:, 1], combined20[:, 1], combined40[:, 1])

# Output the results
print(f"Kruskal-Wallis Test Statistic: {kw_statistic}")
print(f"P-value: {p_value}")


Kruskal-Wallis Test Statistic: 2.8620929874352474
P-value: 0.5811626149696443


Now the arrays for the standard deviation and the flare intensity are created. The hight of the flares is calculated in the loop with the peak to peak value.

In [12]:
#create arrays of the flux hight
std_1d= np.loadtxt('std_1d.txt')
std_5d=   np.loadtxt('std_5d.txt')
std_10d=    np.loadtxt('std_10d.txt')
std_20d=   np.loadtxt('std_20d.txt')
std_40d=   np.loadtxt('std_40d.txt')

flx1 = np.loadtxt('hight_new.txt')
flx5 = np.loadtxt('hight_new.txt')
flx10 = np.loadtxt('hight_new.txt')
flx20 = np.loadtxt('hight_new.txt')
flx40 = np.loadtxt('hight_new.txt')

#match the size of the flx to the size of the std_1d, std_5d, std_10d, std_20d and std_40d separately
if len(flx1) > len(std_1d):
    flx1 = flx1[0:len(std_1d)]
elif len(std_1d) > len(flx1):
    std_1d = std_1d[0:len(flx1)]
else:
    pass

if len(flx5) > len(std_5d):
    flx5 = flx5[0:len(std_5d)]
elif len(std_5d) > len(flx5):
    std_5d = std_5d[0:len(flx5)]
else:
    pass

if len(flx10) > len(std_10d):
    flx10 = flx10[0:len(std_10d)]
elif len(std_10d) > len(flx10):
    std_10d = std_10d[0:len(flx10)]
else:
    pass

if len(flx20) > len(std_20d):
    flx20 = flx20[0:len(std_20d)]
elif len(std_20d) > len(flx20):
    std_20d = std_20d[0:len(flx20)]
else:
    pass

if len(flx40) > len(std_40d):
    flx40 = flx40[0:len(std_40d)]
elif len(std_40d) > len(flx40):
    std_40d = std_40d[0:len(flx40)]
else:
    pass


#combine the flux hight and the standard deviation arrays
combined1 = np.column_stack((flx1, std_1d))
combined5 = np.column_stack((flx5, std_5d))
combined10 = np.column_stack((flx10, std_10d))
combined20 = np.column_stack((flx20, std_20d))
combined40 = np.column_stack((flx40, std_40d))


#sort the arrays by the flux hight starting with the highest value then remove the first 7 rows as well as all nan value rows
combined1 = combined1[combined1[:,0].argsort()[::-1]]
combined1 = combined1[7:]
combined1 = combined1[~np.isnan(combined1).any(axis=1)]

combined5 = combined5[combined5[:,0].argsort()[::-1]]
combined5 = combined5[7:]
combined5 = combined5[~np.isnan(combined5).any(axis=1)]

combined10 = combined10[combined10[:,0].argsort()[::-1]]
combined10 = combined10[7:]
combined10 = combined10[~np.isnan(combined10).any(axis=1)]

combined20 = combined20[combined20[:,0].argsort()[::-1]]
combined20 = combined20[7:]
combined20 = combined20[~np.isnan(combined20).any(axis=1)]

combined40 = combined40[combined40[:,0].argsort()[::-1]]
combined40 = combined40[7:]
combined40 = combined40[~np.isnan(combined40).any(axis=1)]

#save the arrays in txt files
np.savetxt('combined1_hight.txt', combined1)
np.savetxt('combined5_hight.txt', combined5)
np.savetxt('combined10_hight.txt', combined10)
np.savetxt('combined20_hight.txt', combined20)
np.savetxt('combined40_hight.txt', combined40)

In [13]:
# #conduct a multiple regression analysis
# import statsmodels.api as sm


# #perform the multiple regression analysis
# model1 = sm.OLS(combined1[:,1], combined1[:,0]).fit()
# model5 = sm.OLS(combined5[:,1], combined5[:,0]).fit()
# model10 = sm.OLS(combined10[:,1], combined10[:,0]).fit()
# model20 = sm.OLS(combined20[:,1], combined20[:,0]).fit()
# model40 = sm.OLS(combined40[:,1], combined40[:,0]).fit()

# #calculate the correlation coefficients
# print('Correlation coefficient for 1 day:', model1.rsquared)
# print('Correlation coefficient for 5 days:', model5.rsquared)
# print('Correlation coefficient for 10 days:', model10.rsquared)
# print('Correlation coefficient for 20 days:', model20.rsquared)
# print('Correlation coefficient for 40 days:', model40.rsquared)

# #calculate the p-values
# print('p-value for 1 day:', model1.f_pvalue)
# print('p-value for 5 days:', model5.f_pvalue)
# print('p-value for 10 days:', model10.f_pvalue)
# print('p-value for 20 days:', model20.f_pvalue)
# print('p-value for 40 days:', model40.f_pvalue)


Correlation coefficient for 1 day: 0.3872360908553505
Correlation coefficient for 5 days: 0.32589176788197527
Correlation coefficient for 10 days: 0.32368939981268974
Correlation coefficient for 20 days: 0.32492838496941545
Correlation coefficient for 40 days: 0.3421664463289179
p-value for 1 day: 8.506437396945057e-158
p-value for 5 days: 1.8959234765587003e-127
p-value for 10 days: 2.067775856543004e-126
p-value for 20 days: 5.396964117126975e-127
p-value for 40 days: 3.187326882666824e-135
